In [1]:
! pip3 install requests

In [2]:
import requests
import json
import os

In [ ]:
API_KEY = "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"
MAX_CHUNK_SIZE = 3000

In [4]:
def llm_request(prompt: str, api_key: str) -> str:
  try:
    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
        },
        data=json.dumps({
            "model": "deepseek/deepseek-r1-0528:free",
            "messages": [{"role": "user", "content": prompt}],
        })
    )

    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return f"Ошибка {response.status_code}: {response.text}"

  except Exception as e:
    return f"Ошибка при запросе: {str(e)}"

In [ ]:
def process_text(text: str, source: str = "local_text"):
    """Основная функция обработки текста"""
    prompt = f"""Раздели текст ниже на части по следующим правилам:
1. Максимальный размер чанка - {MAX_CHUNK_SIZE} символов
2. Разделяй ТОЛЬКО по границам абзацев (двойной перенос строки)
3. чанк должен включать в себя полностью целые абзацы, недопустимо, чтобы абзац начинался в одном чанке, а заканчивался в другом; если размер абзаца больше, чем {MAX_CHUNK_SIZE} - включаем его целиком в чанк;
3. Не добавляй никаких заголовков, подписей или комментариев
4. Между чанками используй строго разделитель "|||||"
5. Сохраняй оригинальное форматирование
6. Если встречается разделитель вида "Страница 2 из 26" то он игнорируется и дальнейши текст продолжает записываться в текущий чанк до тех пор, пока не будет превышен допустимый размер
7. Разрешено не воспринимать фрагменты не имеющие смысловой нагрузки по типу: "г. Йошкар-Ола, 2024", "Страница 2 из 26" и т.д.

Текст для обработки:
{text}"""

    response = llm_request(prompt, API_KEY)

    if not response:
        return

    chunks = [chunk.strip() for chunk in response.split("|||||") if chunk.strip()]

    for i, chunk in enumerate(chunks, 1):
        chunk_data = {
            "id": f"chunk_{i:02d}",
            "source": source,
            "content": chunk,
            "size": len(chunk)
        }
        with open(f"./chunks/chunk_{i:02d}.json", "w", encoding="utf-8") as f:
            json.dump(chunk_data, f, ensure_ascii=False, indent=2)

In [6]:
! pip3 install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.1 MB/s eta 0:00:00


In [ ]:
import PyPDF2
def read_text_from_pdf(pdf_file_path: str):
    if not os.path.exists(pdf_file_path) or not pdf_file_path.lower().endswith(".pdf"):
        return ""
    with open(pdf_file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''.join(page.extract_text() for page in reader.pages)
    return text

In [1]:
url = 'YOUR PDF URL'
local_filename = 'text.pdf'

import requests

try:
    response = requests.get(url)

    if response.status_code == 200:
        with open(local_filename, 'wb') as f:
            f.write(response.content)
        print(f'Document downloaded successfully and saved as {local_filename}')
    else:
        print(f'Failed to retrieve document. Status code: {response.status_code}')
except Exception as e:
    print(f'An error occurred: {e}')

An error occurred: Invalid URL 'YOUR PDF URL': No scheme supplied. Perhaps you meant https://YOUR PDF URL?


In [14]:
os.makedirs('./chunks', exist_ok=True)
text = read_text_from_pdf('monetaoffer.pdf')

In [15]:
with open('./res.txt', 'w') as f:
    f.write(text)

In [16]:
process_text(text)